In [23]:
from sklearn.preprocessing import StandardScaler
def standard_scaler(df, col_range):
    x = df.iloc[:, col_range].values #returns a numpy array
    gdp_mean = df['GDP per capita (PWT)'].mean()
    gdp_std = df['GDP per capita (PWT)'].std()
    std_scaler = StandardScaler()
    std_scaler.fit(x)
    x_scaled = std_scaler.transform(x)
    df_scaled = df
    df_scaled.iloc[:, col_range] = x_scaled
    return [df_scaled, gdp_mean, gdp_std]

In [24]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn import preprocessing
sns.set()
# pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns')

df = pd.read_csv('../outputs/db.csv', index_col=0)

col_range = list(range(6,16)) + list(range(17,26)) + list(range(27,43))
for i, column in enumerate(df.columns.values):
    print(f"{i} - {column}")

0 - Independent
1 - Sink OFC
2 - Conduit OFC
3 - Code
4 - Country
5 - Year
6 - Population
7 - Population 25 Years Before
8 - Population 25+
9 - Population 25+ / Max Population 25+
10 - Documents
11 - Citable documents
12 - Citations
13 - Self-citations
14 - Citations per Document
15 - H index
16 - Subject
17 - Avg. Adult Schooling
18 - GDP per capita (World Bank)
19 - GDP per capita (PWT)
20 - MAX TAMS
21 - REAL TAMS
22 - Documents / M-TAMS
23 - Documents / R-TAMS
24 - Citations / M-TAMS
25 - Citations / R-TAMS
26 - Gov. Effectiveness
27 - Charges for the use of intellectual property, payments (BoP, current US$)
28 - Charges for the use of intellectual property, receipts (BoP, current US$)
29 - Firms that spend on R&D (% of firms)
30 - High-technology exports (current US$)
31 - Industrial design applications, nonresident, by count
32 - Industrial design applications, resident, by count
33 - Labor force with advanced education (% of total working-age population with advanced education)


In [27]:
#1 para paraíso fiscal
df.loc[df['Sink OFC'] == 'Yes' , 'country_type'] = 1 
#2 para Conduit
df.loc[df['Conduit OFC'] == 'Yes' , 'country_type'] = 2
#0 para quem nõa é nem paraíoso fiscal nem conduit
df.loc[(df['Conduit OFC'] == 'No') & (df['Sink OFC'] == 'No'), 'country_type' ] = 0



In [28]:
def rearrange_db(df, year, step, subjects=['All']):

    years = sorted(df[df['Year'] <= year]['Year'].unique(), reverse=True)
    last_year = years[-1] + step
    df_rearranged = pd.DataFrame()

    for year in years:

        if year >= last_year:
            
            df_economic = df.loc[((df['Year'] == year) & (df['Subject'].isin(subjects))), ['Year', 'Country','GDP per capita (World Bank)', 'GDP per capita (PWT)']]

            df_science = df.loc[((df['Year'] == (year - step)) & (df['Subject'].isin(subjects)))].drop(['Year','Country','GDP per capita (World Bank)', 'GDP per capita (PWT)'], axis=1)

            df_merged = df_economic.join(df_science, how='left')
            df_merged = df_merged[(df_merged['GDP per capita (World Bank)'].notna()) | (df_merged['GDP per capita (PWT)'].notna())]
            df_rearranged = df_rearranged.append(df_merged)

    return df_rearranged
subjects = ['All']
df_rearranged = rearrange_db(df, 2017, 3, subjects)
col_range_rearranged = [2,3] + list(range(8,18)) + list(range(19,26)) + list(range(27,43))
df_rearranged, df_rearranged_mean, df_rearranged_std = standard_scaler(df_rearranged, col_range_rearranged)
df_rearranged.head()



,Year,Country,GDP per capita (World Bank),GDP per capita (PWT),Independent,Sink OFC,Conduit OFC,Code,Population,Population 25 Years Before,Population 25+,Population 25+ / Max Population 25+,Documents,Citable documents,Citations,Self-citations,Citations per Document,H index,Subject,Avg. Adult Schooling,MAX TAMS,REAL TAMS,Documents / M-TAMS,Documents / R-TAMS,Citations / M-TAMS,Citations / R-TAMS,Gov. Effectiveness,"Charges for the use of intellectual property, payments (BoP, current US$)","Charges for the use of intellectual property, receipts (BoP, current US$)",Firms that spend on R&D (% of firms),High-technology exports (current US$),"Industrial design applications, nonresident, by count","Industrial design applications, resident, by count",Labor force with advanced education (% of total working-age population with advanced education),"Patent applications, nonresidents","Patent applications, residents",Research and development expenditure (% of GDP),Researchers in R&D (per million people),Scientific and technical journal articles,Technicians in R&D (per million people),"Trademark applications, nonresident, by count","Trademark applications, resident, by count","Trademark applications, total",country_type
Country code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,2017,Afghanistan,-0.777799,NaN,Yes,No,No,AFG,-0.007339,-0.125711,-0.105832,0.489369,-0.238451,-0.243935,-0.202983,-0.144759,3.350623,-0.612653,All,-1.283119,-0.224734,-0.209655,-0.588325,-0.623231,-0.448224,-0.466850,-1.331396,-0.288592,-0.179035,-0.081122,NaN,NaN,NaN,0.675661,NaN,NaN,NaN,NaN,-0.242821,NaN,NaN,NaN,NaN,0.0
8,2017,Albania,-0.253190,-0.228727,Yes,No,No,ALB,-0.239510,-0.219075,-0.228964,-0.732660,-0.228932,-0.233667,-0.206716,-0.144604,-1.227087,-0.575192,All,0.687342,-0.239795,-0.249413,-0.419851,-0.333952,-0.484808,-0.480269,-0.085558,-0.283749,-0.178998,NaN,-0.292714,-0.223994,-0.183297,-0.831756,-0.257783,-0.206259,NaN,NaN,-0.238824,NaN,-0.357568,-0.27167,-0.243678,0.0
12,2017,Algeria,-0.142470,-0.209815,Yes,No,No,DZA,0.034965,0.017813,0.024130,0.159883,-0.124161,-0.121033,-0.179055,-0.126730,-1.052344,-0.244285,All,0.115273,0.012866,0.026043,-0.309296,-0.322498,-0.442952,-0.453793,-0.477553,-0.256585,-0.179044,NaN,-0.292677,-0.458097,-0.163578,NaN,-0.225808,-0.204653,NaN,NaN,-0.144166,NaN,NaN,NaN,-0.170526,0.0
24,2017,Angola,-0.567130,-0.572345,Yes,No,No,AGO,-0.056319,-0.130201,-0.134564,-0.010461,-0.238604,-0.243814,-0.208218,-0.144796,-1.088152,-0.646993,All,-0.838176,-0.202629,-0.200481,-0.597841,-0.628176,-0.511918,-0.523943,-1.123669,-0.232371,-0.176134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.242854,NaN,NaN,NaN,NaN,0.0
660,2017,Anguilla,NaN,-0.132988,No,Yes,No,AIA,-0.261468,-0.254305,NaN,NaN,-0.240726,-0.246037,-0.208655,-0.144995,-0.287482,-0.712550,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.952229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [29]:
pooled_data = """
data {
  int<lower=0> N; 
  vector[N] x;
  vector[N] y;
}
"""

In [30]:
pooled_parameters = """
parameters {
  vector[2] beta;
  real sigma;
} 
"""
#aqui mudei para o sigma poder ser maior ou menor que zero.


In [ ]:
import pystan
